In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
#Variable 
variable85="rcp85_tg_mean_seasonal"
variable45="rcp45_tg_mean_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_seasonal.nc']

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [46]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

### --------First season - WINTER -  percentiles with Xclim

In [7]:
ds1 = listds[0]
perc45 = ens.ensemble_percentiles(ds1)
perc45

### Transform into dataframe

In [22]:
df45 = perc45.to_dataframe()
df452 = df.reset_index()
df453 = df2.loc[df2["realization"] == 0].dropna()

In [49]:
#df453

In [31]:
#df453.time.unique()

### Split by year periods

In [25]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
dfp452 =  dfp.reset_index()

CPU times: user 2.01 s, sys: 2.41 s, total: 4.42 s
Wall time: 4.42 s


In [48]:
#dfp452

### Transform in Celsius and round

In [47]:
df45C = dfp452.copy()
df45C["tg_mean_p10"] = round(df45C["tg_mean_p10"] -273.15,2)
df45C["tg_mean_p50"] = round(df45C["tg_mean_p50"] -273.15,2)
df45C["tg_mean_p90"] = round(df45C["tg_mean_p90"] -273.15,2)
#df45C

### Pivot Table and Create new columns 

In [50]:
Region1i45 = df45C.drop(columns=["realization"])
#Region1i45

## New columns and rename

In [41]:
df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = ["winter_"+ year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
#df45f

tg_mean_p10               tg_mean_p50                \
time                         0.0    1.0    2.0         0.0    1.0    2.0   
lat       lon                                                              
40.041039 -89.045212       -3.99  -1.56  -1.23       -1.37   0.52   0.95   
          -88.961884       -4.01  -1.58  -1.25       -1.40   0.49   0.93   
          -88.878548       -4.03  -1.59  -1.27       -1.41   0.48   0.92   
          -88.795219       -4.04  -1.61  -1.28       -1.44   0.45   0.90   
          -88.711891       -4.05  -1.60  -1.27       -1.44   0.46   0.91   
...                          ...    ...    ...         ...    ...    ...   
66.623306 -61.629639      -25.07 -20.47 -19.23      -22.20 -17.51 -16.40   
          -61.546310      -25.21 -20.62 -19.40      -22.36 -17.67 -16.56   
          -61.462978      -25.06 -20.46 -19.24      -22.20 -17.51 -16.40   
          -61.379650      -24.98 -20.38 -19.18      -22.13 -17.44 -16.33   
          -61.296322      -24.55 -19.95 -18.78      -21.72 -17.04 -15.93   

                     tg_mean_p90                
time                         0.0    1.0    2.0  
lat       lon                                   
40.041039 -89.045212        0.66   2.59   3.24  
          -88.961884        0.63   2.57   3.21  
          -88.878548        0.61   2.55   3.19  
          -88.795219        0.58   2.52   3.16  
          -88.711891        0.59   2.53   3.17  
...                          ...    ...    ...  
66.623306 -61.629639      -19.30 -14.07 -13.24  
          -61.546310      -19.47 -14.25 -13.42  
          -61.462978      -19.31 -14.08 -13.26  
          -61.379650      -19.24 -14.02 -13.19  
          -61.296322      -18.84 -13.64 -12.81  

[76206 rows x 9 columns]

In [51]:
#df45f

winter_hist_rcp45_p10  winter_t2050_rcp45_p10  \
lat       lon                                                         
40.041039 -89.045212                  -3.99                   -1.56   
          -88.961884                  -4.01                   -1.58   
          -88.878548                  -4.03                   -1.59   
          -88.795219                  -4.04                   -1.61   
          -88.711891                  -4.05                   -1.60   
...                                     ...                     ...   
66.623306 -61.629639                 -25.07                  -20.47   
          -61.546310                 -25.21                  -20.62   
          -61.462978                 -25.06                  -20.46   
          -61.379650                 -24.98                  -20.38   
          -61.296322                 -24.55                  -19.95   

                      winter_t2080_rcp45_p10  winter_hist_rcp45_p50  \
lat       lon                                                         
40.041039 -89.045212                   -1.23                  -1.37   
          -88.961884                   -1.25                  -1.40   
          -88.878548                   -1.27                  -1.41   
          -88.795219                   -1.28                  -1.44   
          -88.711891                   -1.27                  -1.44   
...                                      ...                    ...   
66.623306 -61.629639                  -19.23                 -22.20   
          -61.546310                  -19.40                 -22.36   
          -61.462978                  -19.24                 -22.20   
          -61.379650                  -19.18                 -22.13   
          -61.296322                  -18.78                 -21.72   

                      winter_t2050_rcp45_p50  winter_t2080_rcp45_p50  \
lat       lon                                                          
40.041039 -89.045212                    0.52                    0.95   
          -88.961884                    0.49                    0.93   
          -88.878548                    0.48                    0.92   
          -88.795219                    0.45                    0.90   
          -88.711891                    0.46                    0.91   
...                                      ...                     ...   
66.623306 -61.629639                  -17.51                  -16.40   
          -61.546310                  -17.67                  -16.56   
          -61.462978                  -17.51                  -16.40   
          -61.379650                  -17.44                  -16.33   
          -61.296322                  -17.04                  -15.93   

                      winter_hist_rcp45_p90  winter_t2050_rcp45_p90  \
lat       lon                                                         
40.041039 -89.045212                   0.66                    2.59   
          -88.961884                   0.63                    2.57   
          -88.878548                   0.61                    2.55   
          -88.795219                   0.58                    2.52   
          -88.711891                   0.59                    2.53   
...                                     ...                     ...   
66.623306 -61.629639                 -19.30                  -14.07   
          -61.546310                 -19.47                  -14.25   
          -61.462978                 -19.31                  -14.08   
          -61.379650                 -19.24                  -14.02   
          -61.296322                 -18.84                  -13.64   

                      winter_t2080_rcp45_p90  
lat       lon                                 
40.041039 -89.045212                    3.24  
          -88.961884                    3.21  
          -88.878548                    3.19  
          -88.795219                    3.16  
          -88.711891           

# TODO
- Create for loop for all 4 seasons
- Concat 4DFs info one DF by lat lon

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [ ]:
dsEns85= ens.create_ensemble(files85)
dsEns85

In [ ]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

## Transform into DataFrame

In [ ]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

## Transform in Celsius and round

In [ ]:
df85C = df85.copy()
df85C["tg_mean_p10"] = round(df85C["tg_mean_p10"] -273.15,2)
df85C["tg_mean_p50"] = round(df85C["tg_mean_p50"] -273.15,2)
df85C["tg_mean_p90"] = round(df85C["tg_mean_p90"] -273.15,2)

df85C

## Eliminate 2011, Pivot Table and Create new columns 

In [ ]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

# Merge df45 and 85

In [ ]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

# Extract hist

In [ ]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [ ]:
dftg1

# Merge both hist_50 and get the mean

In [ ]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

In [ ]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

# Merge means with the 45 and 85 DF

In [ ]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [ ]:
dftg_all

In [ ]:
dftg_all.to_feather("tg_mean_annual.feather")

In [ ]:
dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/tg_mean_annual.feather")

In [ ]:
dftg_all 

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [ ]:
"DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [ ]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all(gdf[name].iloc[0], d)+"_tg_mean_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file(short+"_tg_mean_annual.json", driver="GeoJSON")

## Select and open Region (Shapefile)

In [ ]:
region = "DDE_STF_20K_REG_FOR_VUE_S"
             

## Import dataframe with polygons

In [ ]:
dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
#dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-.feather')

In [ ]:
dfpolyshape.head()

In [ ]:
dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
print ("Merged with polygons")
dftp.head()

## Create GeoJson Files

In [ ]:
listTG = []
for tg in dftp[name].unique().tolist():
    df2 = dftp[dftp[name] == tg]
    print (tg)
    if tg != None:
        listTG.append(df2)
listTG

In [ ]:
for tg in listTG:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(tg, crs=crs, geometry=geometry)
    print (gdf[name].iloc[0])
    gdf.to_file(gdf[name].iloc[0]+"_tg_mean_annual.json", driver="GeoJSON")

# Create geojson file for all regions

In [ ]:
geometry = dftp["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
gdf.to_file(short+"_tg_mean_annual.json", driver="GeoJSON")